In [1]:
import os
import polars as pl
from dotenv import load_dotenv
from langchain_community.chat_models import GigaChat
from tqdm.auto import tqdm

In [2]:
load_dotenv()

True

In [3]:
llm = GigaChat(
    verify_ssl_certs=False,
    model="GigaChat-Pro",
    credentials=os.environ.get("GIGACHAT_CREDENTIALS"),
    scope=os.environ.get("GIGACHAT_SCOPE"),
)

In [4]:
# LLM плохо знает часовые пояса, поэтому будет искать наиболее подходящий, используя список регионов
prompt = """Список регионов по часовым поясам:
1) Калининградское время: Калининградская область;
2) Московское время: города федерального значения Москва, Санкт-Петербург, Севастополь, Республика Адыгея, Республика Дагестан, Республика Ингушетия, Кабардино-Балкарская Республика, Республика Калмыкия, Карачаево-Черкесская Республика, Республика Карелия, Республика Коми, Республика Марий Эл, Республика Мордовия, Республика Северная Осетия — Алания, Республика Татарстан, Чеченская республика, Чувашская республика, Краснодарский край, Ставропольский край, Архангельская область, Астраханская область, Белгородская область, Брянская область, Владимирская область, Волгоградская область, Вологодская область, Воронежская область, Ивановская область, Калужская область, Кировская область, Костромская область, Курская область, Ленинградская область, Липецкая область, Московская область, Мурманская область, Нижегородская область, Новгородская область, Орловская область, Пензенская область, Псковская область, Ростовская область, Рязанская область, Саратовская область, Смоленская область, Тамбовская область, Тверская область, Тульская область, Ульяновская область, Ярославская область, Ненецкий автономный округ;
3) Самарское время: Самарская область, Республика Удмуртия;
4) Екатеринбургское время: Республика Башкортостан, Пермский край, Курганская область, Оренбургская область, Свердловская область, Тюменская область, Челябинская область, Ханты-Мансийский автономный округ — Югра и Россия: Ямало-Ненецкий автономный округ;
5) Омское время: Республика Алтай, Алтайский край, Новосибирская область, Омская область, Томская область;
6) Красноярское время: Республика Тыва, Республика Хакасия, Красноярский край, Кемеровская область;
7) Иркутское время: Республика Бурятия, Забайкальский край, Иркутская область;
8) Якутское время: часть Республики Саха (Якутия), Амурская область;
9) Владивостокское время: часть Республики Саха (Якутия), Приморский край, Хабаровский край, Магаданская область, Сахалинская область (кроме Северо-Курильского района), Еврейская автономная область;
10) Среднеколымское время: часть Республики Саха (Якутия), Сахалинская область (только Северо-Курильский район);
11) Камчатское время: Камчатский край, Чукотский автономный округ.
На вход подаётся название региона или города на английском языке. В качестве ответа напиши одно число — номер, к которому название наиболее сильно относится.
< Примеры >
Вход: Moscow
Ответ: 2

Вход: Irkutsk Oblast
Ответ: 7
< Конец примеров >
Вход: {}
Ответ:
"""

In [5]:
regions = pl.read_csv("./data/all_events.csv", columns="region")["region"].unique().sort()

In [6]:
diffs = []
for region in tqdm(regions):
    result = llm.invoke(prompt.format(region))
    diffs.append(int(result.content) + 1)

  0%|          | 0/116 [00:00<?, ?it/s]

In [7]:
time_diffs = pl.DataFrame({"region": regions, "diff": diffs})
time_diffs.head()

region,diff
str,i64
"""Adygeya Republic""",3
"""Altai""",6
"""Altay Kray""",6
"""Amur Oblast""",9
"""Arkhangelsk Oblast""",3


In [8]:
time_diffs.write_parquet("./data/time_diffs.parquet")